In [34]:
import duckdb

In [35]:
def query(sqlString:str)->None:
    
    try:
        return duckdb.query(sqlString).show(max_width = 100000, max_rows = 100000)
    except Exception as e:
        return e

In [8]:
consumption = duckdb.query(r"""


select distinct * from read_csv_auto("C:\Users\MohammedSafiAhmed\Downloads\Consumption Billing Analysis - Consumption Data.csv")

""")

subscription = duckdb.query(r"""


select distinct * from read_csv_auto("C:\Users\MohammedSafiAhmed\Downloads\Consumption Billing Analysis - subscription data.csv")

""")

invoice = duckdb.query(r"""


select distinct * from read_csv_auto("C:\Users\MohammedSafiAhmed\Downloads\Consumption Billing Analysis - Invoice Data.csv")

""")

data = duckdb.query(r"""


select distinct * from read_csv_auto("C:\Users\MohammedSafiAhmed\Downloads\Consumption Billing Analysis - Invoice Item with platform.csv")

""")

newconsumption = duckdb.query(r"""


select distinct * from read_csv_auto("C:\Users\MohammedSafiAhmed\Downloads\Consumption Billing Analysis Data - New Consumption Data.csv")

""")

newsubscription = duckdb.query(r"""


select distinct * from read_csv_auto("C:\Users\MohammedSafiAhmed\Downloads\Consumption Billing Analysis Data - New subscription Data.csv")

""")

newinvoice= duckdb.query(r"""


select distinct * from read_csv_auto("C:\Users\MohammedSafiAhmed\Downloads\Consumption Billing Analysis Data - New Invoice Data.csv")

""")

newdata = duckdb.query(r"""


select distinct * from read_csv_auto("C:\Users\MohammedSafiAhmed\Downloads\Consumption Billing Analysis Data - New Invoice Item Data.csv")

""")

query("select   *  from consumption limit 5")
query("select   *  from subscription limit 5")
query("select   *  from invoice limit 5")
query("select   *  from data limit 5")
query("select   *  from newconsumption limit 5")
query("select   *  from newsubscription limit 5")
query("select   *  from newinvoice limit 5")
query("select   *  from newdata limit 5")

┌──────────┬──────────────────────────────────────┬───────────────┬──────────────┬─────────────────────┐
│ Platform │             Customer UID             │ Billing Month │ Total Amount │ Amount After Margin │
│ varchar  │               varchar                │     date      │    double    │       double        │
├──────────┼──────────────────────────────────────┼───────────────┼──────────────┼─────────────────────┤
│ AZURE    │ fa7c206c-a618-422e-87e5-abb944809d12 │ 2024-01-01    │      7814.98 │         9194.094118 │
│ AZURE    │ 2304b266-8fea-47d5-b923-c34055290745 │ 2024-01-01    │       635.94 │         748.1647059 │
│ AZURE    │ aafcd781-1d8b-474e-ac43-7abbc3503e0f │ 2024-01-01    │    147403.05 │         168330.4412 │
│ AZURE    │ 50324121-0f0f-4bbc-908a-48b45c0cf25a │ 2024-01-01    │     56194.49 │         62779.71765 │
│ AZURE    │ 0b14c51c-9fcc-49f4-8c7d-ea857c4aba63 │ 2024-01-01    │      1138.53 │         1250.017059 │
└──────────┴──────────────────────────────────────┴────

In [36]:
sql = """
select  a.*,b."customerid id"
      from newconsumption a join newsubscription b on a."Subscription Id" = b."Subscription uId"  order by 6
"""

t1 = duckdb.query(sql)
query(sql)



┌──────────┬──────────────────────────────────────┬──────────────────────────────────────┬───────────────┬─────────────────┬─────────────────────┬───────────────┐
│ Platform │             Customer id              │           Subscription Id            │ Billing Month │      Total      │ Amount After Margin │ Customerid Id │
│ varchar  │               varchar                │               varchar                │     date      │     double      │       double        │     int64     │
├──────────┼──────────────────────────────────────┼──────────────────────────────────────┼───────────────┼─────────────────┼─────────────────────┼───────────────┤
│ AZURE    │ 72c27417-145d-400b-952e-893fd1e7646f │ 3263f7d7-baeb-4d01-d3e4-178a0320b9d6 │ 2023-12-01    │       -69988.29 │        -67163.52882 │          7575 │
│ AZURE    │ d4815a57-d729-404d-acf8-c4da4e8b875a │ 228f0367-e13a-4c23-d375-0d29c6f54771 │ 2023-09-01    │       -14423.04 │           -14423.04 │          6912 │
│ AZURE    │ d4815a57-

In [49]:
sql = """select  "customerid id", "customer name",date_trunc('month',cast("order date" as date)) orderdate,sum(total) amountReceived,b.platform,a.invoiceid
                   from newinvoice a join newdata b 
                  on a."invoiceid" = b.invoiceid 
                  --and b.platform = 'AZURE'
                  
                    group by "customerid id",  "customer name",date_trunc('month',cast("order date" as date)),platform,a.invoiceid
                   order by "customerid id",2
                    """

t2 = duckdb.query(sql)

query(sql)



┌───────────────┬───────────────────────────────────────────────────────────┬────────────┬────────────────┬──────────┬───────────┐
│ Customerid id │                       Customer Name                       │ orderdate  │ amountReceived │ Platform │ Invoiceid │
│     int64     │                          varchar                          │    date    │     int128     │ varchar  │   int64   │
├───────────────┼───────────────────────────────────────────────────────────┼────────────┼────────────────┼──────────┼───────────┤
│             9 │ Purple Talk India Private Limited                         │ 2023-06-01 │          23846 │ AZURE    │     92923 │
│             9 │ Purple Talk India Private Limited                         │ 2023-05-01 │         389532 │ GCP      │     90889 │
│             9 │ Purple Talk India Private Limited                         │ 2023-10-01 │           2404 │ GCP      │    101213 │
│             9 │ Purple Talk India Private Limited                         │ 2023-

In [54]:
duckdb.query(f"""
      (select 
      t1.platform,t2."customer name",t1."customerid id",ifnull(sum("Total"),0) "Total Amount" ,sum("Amount After Margin") "Amount After Margin", max("amountReceived") "amountReceived",
      strftime(cast("billing month" as date), '%B %Y') monthname 
      from t1 join t2 
      on t1."customerid id" = t2."customerid id" and cast(t1."billing month" as date) = date_add(date_trunc('month',cast(t2.orderdate as date)),interval (-1) month)
      and t1.platform = t2.platform
      group by t1.platform,t2."customer name",t1."customerid id","billing month"
      order by platform,"billing month" desc,"customerid id")
      
       """).to_csv('../CSVs/new consumption.csv')

In [10]:
# duckdb.query("""select "Customer UID" customerUIDNotMapped from consumption except select "customer uid" from subscription
#       """).to_csv('../CSVs/customersnotmapped.csv')

In [6]:
# t1 = duckdb.query("""
# select  a.*,b.customerid
#       from consumption a join subscription b on a."customer uid" = b."customer uid"  order by 6
# """)
# t1

┌──────────┬──────────────────────────────────────┬───────────────┬──────────────┬─────────────────────┬────────────┐
│ Platform │             Customer UID             │ Billing Month │ Total Amount │ Amount After Margin │ Customerid │
│ varchar  │               varchar                │     date      │    double    │       double        │   int64    │
├──────────┼──────────────────────────────────────┼───────────────┼──────────────┼─────────────────────┼────────────┤
│ AZURE    │ 86fb359e-1360-4ab3-b90d-2a68e8c007b9 │ 2023-12-01    │    354377.86 │         416915.1294 │       4026 │
│ AZURE    │ 86fb359e-1360-4ab3-b90d-2a68e8c007b9 │ 2023-07-01    │    339016.68 │         398843.1529 │       4026 │
│ AZURE    │ 86fb359e-1360-4ab3-b90d-2a68e8c007b9 │ 2023-02-01    │    236730.81 │         278506.8353 │       4026 │
│ AZURE    │ 86fb359e-1360-4ab3-b90d-2a68e8c007b9 │ 2023-01-01    │     223558.7 │         263010.2353 │       4026 │
│ AZURE    │ 86fb359e-1360-4ab3-b90d-2a68e8c007b9 │ 2023

In [7]:
# t2 = duckdb.query("""select  "customer id", date_trunc('month',cast("order date" as date)) orderdate,sum(amount) amountReceived from invoice a join data b on a."invoice id" = b.invoiceid 
#                   and b.platform = 'AZURE'
                  
#                     group by "customer id", date_trunc('month',cast("order date" as date))
#                    order by "customer id",2
#                     """)


In [78]:
# query(f"""
#       (select platform,"customer uid",customerid,ifnull("Total Amount",0) "Total Amount" ,"Amount After Margin","amountReceived",
#       strftime(cast("billing month" as date), '%B %Y') monthname 
#       from t1 join t2 on t1."customerid" = t2."customer id" and cast(t1."billing month" as date) = date_add(date_trunc('month',cast(t2.orderdate as date)),interval (-1) month)
#       order by "billing month" desc,"customer uid")
      
#        """)

┌──────────┬──────────────────────────────────────┬────────────┬──────────────┬─────────────────────┬────────────────┬────────────────┐
│ Platform │             Customer UID             │ Customerid │ Total Amount │ Amount After Margin │ amountReceived │   monthname    │
│ varchar  │               varchar                │   int64    │    double    │       double        │     int128     │    varchar     │
├──────────┼──────────────────────────────────────┼────────────┼──────────────┼─────────────────────┼────────────────┼────────────────┤
│ AZURE    │ 0927ee12-f790-418d-8303-d3055b87ad16 │       9194 │     44806.25 │         52713.23529 │          52713 │ January 2024   │
│ AZURE    │ 0b14c51c-9fcc-49f4-8c7d-ea857c4aba63 │       4879 │      1138.53 │         1250.017059 │           1250 │ January 2024   │
│ AZURE    │ 127d0bca-e9d0-4fcb-bb59-2ac895723837 │       6193 │      9634.55 │         11334.76471 │          11709 │ January 2024   │
│ AZURE    │ 2304b266-8fea-47d5-b923-c3405529074

In [57]:
months = "'January 2024','December 2023','November 2023','October 2023','September 2023','August 2023','July 2023','June 2023','May 2023','April 2023','March 2023','February 2023','January 2023'"

In [77]:
query(f"""select *from
      (select platform,"customer uid",customerid,ifnull("Total Amount",0) "Total Amount" ,"Amount After Margin","amountReceived",
      strftime(cast("billing month" as date), '%B %Y') monthname 
      from t1 join t2 on t1."customerid" = t2."customer id" and cast(t1."billing month" as date) = date_add(date_trunc('month',cast(t2.orderdate as date)),interval (-1) month)
      order by "billing month" desc,"customer uid")
      pivot ( sum("Total Amount") totalAmount, sum("Amount After Margin") "Amount After Margin", sum("amountReceived") "amountReceived" for "monthname" in ({months}))
       """)
# .to_csv('../CSVs/azure_consumption_pivot.csv')

┌──────────┬──────────────────────────────────────┬────────────┬──────────────────────────┬──────────────────────────────────┬─────────────────────────────┬───────────────────────────┬───────────────────────────────────┬──────────────────────────────┬───────────────────────────┬───────────────────────────────────┬──────────────────────────────┬──────────────────────────┬──────────────────────────────────┬─────────────────────────────┬────────────────────────────┬────────────────────────────────────┬───────────────────────────────┬─────────────────────────┬─────────────────────────────────┬────────────────────────────┬───────────────────────┬───────────────────────────────┬──────────────────────────┬───────────────────────┬───────────────────────────────┬──────────────────────────┬──────────────────────┬──────────────────────────────┬─────────────────────────┬────────────────────────┬────────────────────────────────┬───────────────────────────┬────────────────────────┬──────────────────

In [75]:
query(f"""select string_agg(concat('''',monthname,''''),',' order by "billing month" desc) month1 from
      (select --platform,"customer uid",customerid,"Total Amount","Amount After Margin","amountReceived",
      distinct "billing month",
       strftime(cast("billing month" as date), '%B %Y') monthname 
      from t1 join t2 on t1."customerid" = t2."customer id" and cast(t1."billing month" as date) = date_add(date_trunc('month',cast(t2.orderdate as date)),interval (-1) month)
      order by "billing month" desc,"customer uid")
      --pivot ( sum("Total Amount") totalAmount, sum("Amount After Margin") "Amount After Margin", sum("amountReceived") "amountReceived" for "monthname" in ({months}))
       """)

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                          month1                                                                                          │
│                                                                                         varchar                                                                                          │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 'January 2024','December 2023','November 2023','October 2023','September 2023','August 2023','July 2023','June 2023','May 2023','April 2023','March 2023','February 2023','January 2023' │
└──────────────────────────────────────────────────────

In [1]:
cxn.close(0)

NameError: name 'cxn' is not defined